In [242]:
from qiskit import *
from qiskit.circuit.library import *
from qiskit.primitives import Sampler
import numpy as np 
from qiskit.extensions import *
from qiskit.algorithms import HamiltonianPhaseEstimation
from qiskit.quantum_info.operators import Operator, Pauli

A = [
        [1,0],
        [0,-1]
    ]

A = [
        [0,0,5,4],
        [0,0,1,2],
        [5,1,0,0],
        [4,2,0,0]
    ]

precision = 3
qreg = QuantumRegister(precision)
qregb = QuantumRegister(1)
creg = ClassicalRegister(precision)

QC = QuantumCircuit(qreg, qregb, creg)
QC.x(-1)
pe = HamiltonianPhaseEstimation(5, sampler=Sampler())
z = pe.estimate(randHamiltonian)
print (z)

{'most_likely_eigenvalue': -6.75, 'phase': 0.625}


In [243]:
def binaryToPhase(binary):
    binary=list(binary[::-1])
    length = len(binary)
    res=0
    
    for i in range(length):
        digit=int(binary.pop())
        if digit==1:
            res=res+pow(2,i)  
    return res

# job = Sampler().run(QC, shots=100)
# for k,v in job.result().quasi_dists[0].items():
#     print (bin(k)[2:], v, binaryToPhase(bin(k)[2:]))

In [244]:
import itertools
import functools as ft
from qiskit.quantum_info import SparsePauliOp

## ref : https://obliviateandsurrender.github.io/blogs/vqe.html
def decompose_ham_to_pauli(H):
    n = int(np.log2(len(H)))
    N = 2 ** n
    
    sI = np.eye(2, 2, dtype=complex)
    sX = np.array([[0, 1], [1, 0]], dtype=complex)
    sZ = np.array([[1, 0], [0,-1]], dtype=complex)
    sY = complex(0,-1)*np.matmul(sZ,sX)
    paulis = [sI, sX, sY, sZ]
    paulis_label = ['I', 'X', 'Y', 'Z']
    obs = []
    coeffs = []
    matrix = []
    for term in itertools.product(paulis, repeat=n):
        matrices = [pauli for pauli in term]
        coeff = np.trace(ft.reduce(np.kron, matrices) @ H) / N 
        coeff = np.real_if_close(coeff).item()
        # Hilbert-Schmidt-Product
        if not np.allclose(coeff, 0): 
            coeffs.append(coeff)
            obs.append(''.join([paulis_label[[i for i, x in enumerate(paulis)
            if np.all(x == t)][0]] for idx, t in enumerate(reversed(term))]))
            matrix.append(ft.reduce(np.kron, matrices))

    return obs, coeffs , matrix

o, amplitudes, _ = decompose_ham_to_pauli(A)
randHamiltonian = SparsePauliOp.from_list(list(zip(o, amplitudes)))